# Minor Flood Frequency

Writing words, still lots to do.

In [21]:
%run setup.ipynb
import calendar

In [23]:
output_dir

PosixPath('/Users/jfiedler/Library/CloudStorage/GoogleDrive-jfiedler@hawaii.edu/Shared drives/PI Indicators 2025/SL Indicators/Hawaii_Region_Output')

In [4]:
# load the data
# SL_daily_max = hourly_data.resample(time='D').max()

# Load the data with explicit decode_timedelta parameter
SL_daily_max = xr.open_dataset(data_dir / 'SL_daily_max.nc', decode_timedelta=True)
SL_daily_max.close()

hourly_data = xr.open_dataset(data_dir / 'SL_hourly_data.nc', decode_timedelta=True)
hourly_data.close()



In [7]:
SL_daily_max

<xarray.Dataset> Size: 2MB
Dimensions:          (time: 15341, station_id: 7)
Coordinates:
  * station_id       (station_id) <U7 196B '1617760' '1617433' ... '1619910'
  * time             (time) datetime64[ns] 123kB 1983-01-01 ... 2024-12-31
Data variables: (12/15)
    sea_level        (time, station_id) float64 859kB ...
    sea_level_MHHW   (time, station_id) float64 859kB ...
    day              (time) int64 123kB ...
    month            (time) int64 123kB ...
    year             (time) int64 123kB ...
    year_storm       (time) int64 123kB ...
    ...               ...
    lon              (station_id) float64 56B ...
    MHHW             (station_id) float64 56B ...
    MSL              (station_id) float64 56B ...
    MLLW             (station_id) float64 56B ...
    flood_day        (time, station_id) bool 107kB ...
    threshold        (station_id) float64 56B ...

In [14]:
# threshold_name = SL_daily_max['threshold'].attr['long_name']
threshold = SL_daily_max['threshold']
threshold.attrs['units']


'cm'

In [26]:
SL_daily_max

station_id = SL_daily_max['station_id'].values[0]
station_name = SL_daily_max['station_name'].sel(station_id=station_id).item()
flood_threshold = SL_daily_max['threshold'].sel(station_id=station_id).item()
threshold_units = SL_daily_max['threshold'].attrs['units']
threshold_name = SL_daily_max['threshold'].attrs['long_name']

# Make a dataframe with years as rows and months as columns
df = SL_daily_max['flood_day'].sel(station_id=station_id).to_dataframe()
df = df.reset_index()
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df = df.drop(columns='time')

# sum the number of flood days in each month for each year
df = df.groupby(['year', 'month']).sum()

# pivot the table
df = df.pivot_table(index='year', columns='month', values='flood_day')

# rename months to month names
df.columns = calendar.month_abbr[1:]

# add a column for the total number of flood days in each year
df['Annual'] = df.sum(axis=1)



# add a row for the total number of flood days in each month, normalized the total flood days (last row, last column)
df.loc['Monthly Total (%)'] = df.sum()
df.loc['Monthly Total (%)'] = 100*df.loc['Monthly Total (%)']/df.loc['Monthly Total (%)', 'Annual']


days_exceeding_str = 'Days Exceeding ' + str(flood_threshold) + ' ' + threshold_units + ' above MHHW'
tableTitle = station_name + days_exceeding_str + ' <br>(' + threshold_name + ')'
# Apply background gradient and add a title with larger text
styled_df = df.style.background_gradient(
    cmap='Blues', axis=None, subset=(df.index[:-1], df.columns[:-1])
).format("{:.0f}").background_gradient(
    cmap='Purples', axis=None, subset=(df.index[-1], df.columns[:-1])
).format("{:.0f}").set_caption(tableTitle).set_table_styles([
    {
        'selector': 'caption',
        'props': [('font-size', '16px'), ('margin-bottom', '10px')]
    }
])

styled_df


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
year,,,,,,,,,,,,,
1983,4,0,0,0,0,0,0,1,5,0,0,4,14
1984,1,0,0,0,0,0,0,5,3,0,0,0,9
1985,0,0,0,0,0,0,0,0,0,0,0,0,0
1986,0,0,0,0,0,0,0,0,0,0,0,3,3
1987,5,0,0,0,0,0,0,2,0,0,7,1,15
1988,1,5,0,0,0,0,3,2,0,3,2,0,16
1989,0,0,0,0,0,0,0,0,0,0,0,0,0
1990,0,0,0,0,0,0,0,0,0,0,0,2,2
1991,3,0,0,0,0,0,0,2,0,0,2,2,9


In [34]:
#make a pretty pdf of the table with great_tables
from great_tables import GT,html

dfGT = df.copy()
dfGT['year'] = df.index

# put the year column first
cols = dfGT.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfGT = dfGT[cols]
dfGT.reset_index(drop=True, inplace=True)



# Create a GreatTable object
table = (GT(dfGT)
.fmt_number(columns=calendar.month_abbr[1:13], decimals=0)
.fmt_number(columns=['Annual'], decimals=0)
.tab_header(title = days_exceeding_str, subtitle = station_name)
.data_color(domain = [0,30],
            columns=calendar.month_abbr[1:13],
            rows = list(range(len(dfGT)-1)),
            palette=["white", "#31B1E3"])  # Adjusted blue
.data_color(domain = [0,30],
            columns=calendar.month_abbr[1:13],
            rows = [-1],
            palette=["white", "purple"])
.tab_source_note(f'Data Source: NOAA CO-OPS Hourly Water Level, {threshold_name}')
)

# save table as png
tableName = station_name + '_flood_days_intra_annual.png'
savePath = os.path.join(output_dir, tableName)

# replace any commas or spaces with underscores
# savePath = savePath.replace(' ', '_')
table.save(savePath, scale=2)

# Load Image
from IPython.display import Image
imgTable = Image(filename=savePath)

# Glue the image with a name
glue("imgTable", imgTable, display=False)

table



GT(_tbl_data=                 year        Jan      Feb       Mar       Apr       May  \
0                1983   4.000000  0.00000  0.000000  0.000000  0.000000   
1                1984   1.000000  0.00000  0.000000  0.000000  0.000000   
2                1985   0.000000  0.00000  0.000000  0.000000  0.000000   
3                1986   0.000000  0.00000  0.000000  0.000000  0.000000   
4                1987   5.000000  0.00000  0.000000  0.000000  0.000000   
5                1988   1.000000  5.00000  0.000000  0.000000  0.000000   
6                1989   0.000000  0.00000  0.000000  0.000000  0.000000   
7                1990   0.000000  0.00000  0.000000  0.000000  0.000000   
8                1991   3.000000  0.00000  0.000000  0.000000  0.000000   
9                1992   3.000000  2.00000  0.000000  0.000000  0.000000   
10               1993   2.000000  1.00000  0.000000  1.000000  1.000000   
11               1994   0.000000  0.00000  0.000000  0.000000  0.000000   
12               1995   0.000000  0.00000  0.000000  0.000000  0.000000   
13               1996   1.000000  1.00000  0.000000  0.000000  0.000000   
14               1997   2.000000  0.00000  0.000000  0.000000  0.000000   
15               1998   2.000000  0.00000  0.000000  0.000000  0.000000   
16               1999   0.000000  0.00000  0.000000  0.000000  0.000000   
17               2000   0.000000  0.00000  0.000000  0.000000  0.000000   
18               2001   0.000000  0.00000  0.000000  0.000000  0.000000   
19               2002   2.000000  0.00000  0.000000  0.000000  0.000000   
20               2003  10.000000  1.00000  0.000000  0.000000  0.000000   
21               2004   0.000000  0.00000  0.000000  0.000000  3.000000   
22               2005   4.000000  3.00000  1.000000  0.000000  0.000000   
23               2006   1.000000  1.00000  0.000000  0.000000  0.000000   
24               2007   1.000000  0.00000  0.000000  0.000000  0.000000   
25               2008   0.000000  0.00000  0.000000  0.000000  0.000000   
26               2009   4.000000  0.00000  0.000000  0.000000  2.000000   
27               2010   2.000000  0.00000  0.000000  0.000000  0.000000   
28               2011   2.000000  0.00000  2.000000  0.000000  0.000000   
29               2012   0.000000  0.00000  0.000000  0.000000  1.000000   
30               2013   0.000000  0.00000  0.000000  0.000000  0.000000   
31               2014   3.000000  0.00000  0.000000  0.000000  0.000000   
32               2015   1.000000  0.00000  0.000000  0.000000  0.000000   
33               2016   0.000000  0.00000  0.000000  0.000000  1.000000   
34               2017   4.000000  7.00000  0.000000  3.000000  6.000000   
35               2018   7.000000  2.00000  0.000000  0.000000  1.000000   
36               2019   0.000000  0.00000  0.000000  0.000000  1.000000   
37               2020   7.000000  4.00000  4.000000  0.000000  5.000000   
38               2021  12.000000  6.00000  0.000000  2.000000  3.000000   
39               2022   6.000000  1.00000  0.000000  0.000000  2.000000   
40               2023   0.000000  2.00000  0.000000  0.000000  0.000000   
41               2024  11.000000  3.00000  2.000000  2.000000  1.000000   
42  Monthly Total (%)  13.359788  5.15873  1.190476  1.058201  3.571429   

          Jun        Jul        Aug       Sep       Oct        Nov        Dec  \
0    0.000000   0.000000   1.000000  5.000000  0.000000   0.000000   4.000000   
1    0.000000   0.000000   5.000000  3.000000  0.000000   0.000000   0.000000   
2    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   0.000000   
3    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   3.000000   
4    0.000000   0.000000   2.000000  0.000000  0.000000   7.000000   1.000000   
5    0.000000   3.000000   2.000000  0.000000  3.000000   2.000000   0.000000   
6    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   0.000000   
7    0.000000   0.000000   0.000000  0

```{glue:figure} imgTable
:name: "table"
:align: center
:scale: 50%
Table of sea level records exceeding a given threshold. The data are organized into years, and months. Each 'event' is considered a 'flooding day,' where at least one hour of that day exceeds the threshold. The monthly percentage shown at the bottom of the table is relative to the total number of flooding days (for all months and years in the record). 
```